In [3]:
from pathlib import Path
import shutil
import subprocess

In [4]:
IWASM = Path(shutil.which("iwasm"))

PY2WASM = Path(shutil.which("py2wasm"))

PYTHON_COOKBOOK_EXAMPLES_HOME = Path.cwd().joinpath("./cookbook/src")
assert PYTHON_COOKBOOK_EXAMPLES_HOME.exists(), "use `git submodule update` to fetch cookbook"


In [27]:
def compile_py_2_wasm(py2wasm_bin: Path, py_file: Path) -> Path:
    """
    print both stdout and stderr when running the command and meeting an error
    """
    wasm_file = py_file.with_suffix(".wasm")
    cmd = f"{py2wasm_bin} {py_file} -o {wasm_file}"
    try:
        subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    except subprocess.CalledProcessError as e:
        print(e.stderr)
        raise e

    return wasm_file

def execute_py(py_file: Path) -> subprocess.CompletedProcess:
    cmd = f"python {py_file}"
    return subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)

def execute_wasm(iwasm_bin: Path, wasm_file: Path) -> subprocess.CompletedProcess:
    cmd = f"{iwasm_bin} {wasm_file}"
    try:
        p = subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    except subprocess.CalledProcessError as e:
        print(e.stdout)
        print(f"{'-' * 50}")
        print(e.stderr)
        raise e

    return p

def compare_result(expected: str, actual: str) -> bool:
    if expected != actual:
        print(f"{'<' * 50}")
        print(f"{expected}")
        print(f"{'-' * 50}")
        print(f"{actual}")
        print(f"{'>' * 50}")
        return False

    return True


In [28]:
py_file = Path("./example.py")
wasm_file = compile_py_2_wasm(PY2WASM, py_file)

p = execute_wasm(IWASM, wasm_file)
output_from_wasm = p.stdout

p = execute_py(py_file)
output_from_py = p.stdout

compare_result(output_from_py, output_from_wasm)

True